# Data Engineer - ETL


## Imports

Import any additional libraries you may need here.


In [1]:
import glob
import pandas as pd
from datetime import datetime

## Extract


### JSON Extract Function

This function will extract JSON files.


In [12]:
def extract_from_json(file_to_process):
    dataframe = pd.read_json(file_to_process)
    return dataframe

def extract_from_csv(file_to_process):
    dataframe = pd.read_csv(file_to_process)
    return dataframe

## Extract Function

Define the extract function that finds JSON file `bank_market_cap_1.json` and calls the function created above to extract data from them. Store the data in a `pandas` dataframe. Use the following list for the columns.


In [13]:
columns=['Name','Market Cap (US$ Billion)']

In [20]:
def extract():
    extracted_data_list = []
    for jsonfile in glob.glob("*.json"):
        extracted_data_list.append(extract_from_json(jsonfile))

    if not extracted_data_list:
        return pd.DataFrame(columns=['Name', 'Market Cap (US$ Billion)'])
    else:
        return pd.concat(extracted_data_list, ignore_index=True)

In [21]:
def exchange_rates(currency):
    exchange_rate = extract_from_csv('exchange_rates.csv')
    exchange_rate.rename(columns = {'Unnamed: 0':'Currency'}, inplace = True)
    exchange_rate = exchange_rate.set_index('Currency')
    return exchange_rate.loc[[currency]]
    
exchange_rates('GBP')


,Rates
Currency,
GBP,0.732398


## Transform

In [22]:
def transform(bank_market_cap, exchange_rates):
    bank_market_cap['Market Cap (GBP$ Billion)'] = bank_market_cap['Market Cap (US$ Billion)'].mul(float(exchange_rates['Rates']))
    bank_market_cap = bank_market_cap.drop('Market Cap (US$ Billion)', axis=1)
    return bank_market_cap

transform(extract(), exchange_rates('GBP'))

C:\Users\ADMINM~1\AppData\Local\Temp/ipykernel_12692/3885177868.py:2: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  bank_market_cap['Market Cap (GBP$ Billion)'] = bank_market_cap['Market Cap (US$ Billion)'].mul(float(exchange_rates['Rates']))


,Name,Market Cap (GBP$ Billion)
0,JPMorgan Chase,286.319444
1,Industrial and Commercial Bank of China,252.834188
2,Bank of America,238.271911
3,Wells Fargo,225.588235
4,China Construction Bank,188.518621
...,...,...
100,Ping An Bank,27.826013
101,Standard Chartered,27.332377
102,United Overseas Bank,25.727692
103,QNB Group,24.579291


## Load

In [23]:
def load(target_file, data_to_load):
        data_to_load.to_csv(target_file, index=False) 

## Logging Function


Write the logging function <code>log</code> to log your data:


In [24]:
def log(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S' # Year-Monthname-Day-Hour-Minute-Second
    now = datetime.now() # get current timestamp
    timestamp = now.strftime(timestamp_format)
    with open("logfile.txt","a") as f:
        f.write(timestamp + ',' + message + '\n')

## Running the ETL Process


In [25]:
log("ETL Job Started")
log("Extract phase Started")

### Extract


In [26]:
# Call the function
extracted_data = extract()
# Print the rows
extracted_data.head()

,Name,Market Cap (US$ Billion)
0,JPMorgan Chase,390.934
1,Industrial and Commercial Bank of China,345.214
2,Bank of America,325.331
3,Wells Fargo,308.013
4,China Construction Bank,257.399


Log the data as <code>"Extract phase Ended"</code>


In [27]:
log("Extract phase Ended")

### Transform


Log the following  <code>"Transform phase Started"</code>


In [28]:
log("Transform phase Started")

In [29]:
# Call the function
transformed_data = transform(extracted_data, exchange_rates('GBP'))
# Print the first 5 rows
transformed_data.head()

C:\Users\ADMINM~1\AppData\Local\Temp/ipykernel_12692/3885177868.py:2: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  bank_market_cap['Market Cap (GBP$ Billion)'] = bank_market_cap['Market Cap (US$ Billion)'].mul(float(exchange_rates['Rates']))


,Name,Market Cap (GBP$ Billion)
0,JPMorgan Chase,286.319444
1,Industrial and Commercial Bank of China,252.834188
2,Bank of America,238.271911
3,Wells Fargo,225.588235
4,China Construction Bank,188.518621


Log your data <code>"Transform phase Ended"</code>


In [30]:
log("Transform phase Ended")

### Load


Log the following `"Load phase Started"`.


In [31]:
log("Load phase Started")

Call the load function


In [32]:
load('market_cap.csv', transformed_data)

Log the following `"Load phase Ended"`.


In [33]:
log("Load phase Ended")